In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
import joblib
from tqdm import tqdm
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from preprocessing.noise_reduction import NoiseReducer
from preprocessing.silence_removal import SilenceRemover
from preprocessing.speech_filter import SpeechFilter
from preprocessing.identity import Identity


from audio import Audio

original_metadata_path = os.path.join(".", "data", "original_data_labeled.tsv")
filtered_metadata_path = os.path.join(".", "data", "filtered_data_labeled.tsv")
audio_dir = os.path.join(".", "data", "filtered_clips")

### Pipeline

In [16]:

folder_path = 'trials/features/all_classes_155'
X_train_loaded = joblib.load(os.path.join(folder_path, 'X_train.joblib'))
X_test_loaded = joblib.load(os.path.join(folder_path, 'X_test.joblib'))
y_train_loaded = joblib.load(os.path.join(folder_path, 'y_train.joblib'))
y_test_loaded = joblib.load(os.path.join(folder_path, 'y_test.joblib'))

print(X_train_loaded.shape, X_test_loaded.shape, y_train_loaded.shape, y_test_loaded.shape)

(145504, 157) (36376, 157) (145504,) (36376,)


In [ ]:
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def find_best_feature_combination(X, y, feature_indices):
    """
    Find the best feature combination for classification, including non-consecutive combinations, and store all accuracies.

    Args:
        X (numpy.ndarray): Feature vectors.
        y (numpy.ndarray): Labels.
        feature_indices (list of tuples): List of (start, end) indices for each feature vector.

    Returns:
        dict: Best combination, accuracy, feature indices, and all accuracies.
    """
    best_accuracy = 0
    best_combination = None
    best_features = None
    all_accuracies = []

    # Generate all possible feature combinations
    print("Generating all possible feature combinations...")
    print(f"Feature indices: {feature_indices}")
    total_combinations = sum(len(list(combinations(feature_indices, r))) for r in range(1, len(feature_indices) + 1))
    print(f"Total combinations to evaluate: {total_combinations}")
    with tqdm(total=total_combinations, desc="Evaluating feature combinations") as pbar:
        for r in range(1, len(feature_indices) + 1):
            for combination in combinations(feature_indices, r):
                # Extract features for the current combination
                selected_features = []
                for start, end in combination:
                    selected_features.append(X[:, start:end])
                X_combined = np.hstack(selected_features)

                # Split data into training and testing sets
                X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

                # Train a classifier
                clf = RandomForestClassifier(random_state=42)
                clf.fit(X_train, y_train)

                # Evaluate accuracy
                y_pred = clf.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)

                # Store the accuracy and combination
                all_accuracies.append({"combination": combination, "accuracy": accuracy})

                # Update the best combination if current accuracy is higher
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_combination = combination
                    best_features = X_combined

                pbar.update(1)

    return {
        "best_combination": best_combination,
        "best_accuracy": best_accuracy,
        "best_features": best_features,
        "all_accuracies": all_accuracies
    }

feature_indices = [(0, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 155)]  # Example feature vector indices
mapping = {
    "mfcc": (0, 75),
    "fundamental_freq": (75, 76),
    "jitter": (76, 77),
    "alpha_ratio": (77, 78),
    "cpps": (78, 79),
    "pitch_range": (79, 80),
    "hfcc": (80, 155)
}
result = find_best_feature_combination(X_train_loaded, y_train_loaded, feature_indices)
print("Best Combination:", result["best_combination"])
print("Best Accuracy:", result["best_accuracy"])
print("All Accuracies:", result["all_accuracies"])


Generating all possible feature combinations...
Feature indices: [(0, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 155)]
Total combinations to evaluate: 127


Evaluating feature combinations: 100%|██████████| 127/127 [6:18:25<00:00, 178.78s/it]  

Best Combination: ((0, 75), (78, 79), (79, 80), (80, 155))
Best Accuracy: 0.8642314697089447
All Accuracies: [{'combination': ((0, 75),), 'accuracy': 0.8330297927906257}, {'combination': ((75, 76),), 'accuracy': 0.5194666849936428}, {'combination': ((76, 77),), 'accuracy': 0.5251365932442185}, {'combination': ((77, 78),), 'accuracy': 0.5088828562592351}, {'combination': ((78, 79),), 'accuracy': 0.5068554345211504}, {'combination': ((79, 80),), 'accuracy': 0.5046905604618398}, {'combination': ((80, 155),), 'accuracy': 0.8613449709631972}, {'combination': ((0, 75), (75, 76)), 'accuracy': 0.8299714786433456}, {'combination': ((0, 75), (76, 77)), 'accuracy': 0.8337170543967561}, {'combination': ((0, 75), (77, 78)), 'accuracy': 0.8335796020755301}, {'combination': ((0, 75), (78, 79)), 'accuracy': 0.8368784577849558}, {'combination': ((0, 75), (79, 80)), 'accuracy': 0.8327205250678671}, {'combination': ((0, 75), (80, 155)), 'accuracy': 0.8631318511391362}, {'combination': ((75, 76), (76, 77)

In [35]:
# Sort all accuracies in descending order
sorted_accuracies = sorted(result["all_accuracies"], key=lambda x: x["accuracy"], reverse=True)

# Write the sorted accuracies to a text file in tabular form with improved formatting
with open("sorted_accuracies.txt", "w") as f:
    # Write the header with borders
    f.write("|" + "=" * 72 + "|" + "=" * 12 + "|\n")
    f.write(f"| {'Combination':<70} | {'Accuracy':<10} |\n")
    f.write("|" + "=" * 72 + "|" + "=" * 12 + "|\n")
    
    # Write each combination and its accuracy with borders
    for entry in sorted_accuracies:
        combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
        f.write(f"| {combination_str:<70} | {entry['accuracy']:<10.6f} |\n")
    f.write("|" + "=" * 72 + "|" + "=" * 12 + "|\n")

# Display the sorted combinations with their accuracies
for entry in sorted_accuracies:
    combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
    print(f"Combination: {combination_str}, Accuracy: {entry['accuracy']}")

Combination: mfcc, cpps, pitch_range, hfcc, Accuracy: 0.8642314697089447
Combination: mfcc, fundamental_freq, jitter, alpha_ratio, cpps, pitch_range, hfcc, Accuracy: 0.8641971066286382
Combination: mfcc, jitter, alpha_ratio, pitch_range, hfcc, Accuracy: 0.8639909281467991
Combination: mfcc, fundamental_freq, alpha_ratio, pitch_range, hfcc, Accuracy: 0.8637503865846534
Combination: mfcc, alpha_ratio, cpps, pitch_range, hfcc, Accuracy: 0.8637503865846534
Combination: mfcc, alpha_ratio, cpps, hfcc, Accuracy: 0.8636129342634273
Combination: mfcc, fundamental_freq, cpps, hfcc, Accuracy: 0.8635785711831209
Combination: mfcc, jitter, alpha_ratio, hfcc, Accuracy: 0.8634754819422013
Combination: mfcc, hfcc, Accuracy: 0.8631318511391362
Combination: mfcc, jitter, pitch_range, hfcc, Accuracy: 0.8631318511391362
Combination: mfcc, alpha_ratio, hfcc, Accuracy: 0.8629943988179101
Combination: mfcc, fundamental_freq, alpha_ratio, hfcc, Accuracy: 0.8629600357376035
Combination: mfcc, jitter, alpha_rat

In [ ]:
from playsound import playsound

def play_sound(file_path):
    """
    Play a sound file.

    Args:
        file_path (str): Path to the sound file.
    """
    try:
        playsound(file_path)
        playsound(file_path)
    except Exception as e:
        print(f"Error playing sound: {e}")

play_sound("../sound/done.mp3")